In [ ]:
!pip install sentinelhub

     |████████████████████████████████| 204kB 9.2MB/s 
     |████████████████████████████████| 133kB 16.0MB/s 
     |████████████████████████████████| 7.6MB 11.6MB/s 
     |████████████████████████████████| 6.6MB 29.1MB/s 
     |████████████████████████████████| 133kB 46.6MB/s 
     |████████████████████████████████| 81kB 8.4MB/s 
     |████████████████████████████████| 51kB 6.3MB/s 
  Created wheel for sentinelhub: filename=sentinelhub-3.3.1-cp37-none-any.whl size=210148 sha256=d7628ba6b914b843ea9faf067a5a557a818f8724f067331b5ce4841855d332e4
  Stored in directory: /root/.cache/pip/wheels/40/81/42/eed5d100149b40f3aa73d1724b50bd4f78b87a32fb0a3e8a16
  Created wheel for utm: filename=utm-0.7.0-cp37-none-any.whl size=6110 sha256=f7f05cf50256675af895c698985adb5ff7fa78c7a5d557c2980c4eb5e89ec82b
  Stored in directory: /root/.cache/pip/wheels/d3/a0/c4/93c7535fe705e70d2376505fd2683f91524a50a944a15212d9
  Created wheel for stringcase: filename=stringcase-1.2.0-cp37-none-any.whl size=3588 sha256=

# Collect Satellite images and masks of the farmers location from multiple timestamps

Our assignment is to detect, map and analyze deforestation in regions around India (more specifically, West Bengal). In order to be able to analyze deforestation, we must first be able to recognize afforestation from different periods. For that reason, we have satellite images from different periods ( Time Series satellite images ) and different spectral emissions ( Multispectral images), such as RGB and NDVI.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Spectral emissions
Different channels give different information, and the combinations of channels reveal more information about those satellite images.

#### RGB 
The colors in an image will depend on what kind of light the satellite instrument measured. True-color images use visible light—red, green and blue wavelengths(RGB) thus the colors are similar to what a person would see from space.

The true color product maps Sentinel-2 band values B04, B03, and B02 to the R, G, B components

#### NDVI
The NDVI is a basic yet efficient metric for assessing green vegetation that is well-known and extensively utilized.
It compares green leaf scattering in near-infrared wavelengths to chlorophyll absorption in red wavelengths to normalize green leaf scattering. 

The normalized difference vegetation index, abbreviated NDVI, is defined as
<img src="markdown_images\ndvi_calculations.png">

In [ ]:
# python_script. In jupyter notebook, yoo can delete this cel and just import it

# connector.py
from sentinelhub import SHConfig
# from dotenv import load_dotenv
import os

# load_dotenv()
config = SHConfig()
config.sh_client_id = '0bd9981e-9f55-48f8-a564-f5db87422602' 
config.sh_client_secret = '2#?ILN5#vU&0y6+{vm*]:^+Fqae^ro]:!>I1ka8+'

if config.sh_client_id == '' or config.sh_client_secret == '':
    print("Warning! To use Sentinel Hub Process API, please provide the credentials (client ID and client secret).")
else:
    print("Succefully connected!")

# helper.py 
import numpy as np
import matplotlib.pyplot as plt
from sentinelhub import SentinelHubRequest, BBox, CRS, DataCollection, bbox_to_dimensions
from sentinelhub.constants import MimeType


def plot_image(image, factor=1/255, clip_range=(0, 1), **kwargs):
    """
    Utility function for plotting RGB images.
    """
    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(15, 15))
    if clip_range is not None:
        ax.imshow(np.clip(image * factor, *clip_range), **kwargs)
    else:
        ax.imshow(image * factor, **kwargs)
    ax.set_xticks([])
    ax.set_yticks([])


def send_request(evalscript, input_data, coords, config, save_data, data_folder='../data/images'):
    bbox = BBox(bbox=coords, crs=CRS.WGS84)
    request = SentinelHubRequest(data_folder=data_folder, evalscript=evalscript, input_data=input_data, bbox=bbox, config=config, size=[512, 512], responses=[SentinelHubRequest.output_response('default', MimeType.TIFF)])
    response = request.get_data(save_data=save_data)[0]
    return response


simple_request = lambda eval_script, coords, config, save_data, **kwargs: send_request(
    eval_script, 
    [SentinelHubRequest.input_data(data_collection=DataCollection.SENTINEL2_L2A, time_interval=('2018-01-01', '2019-01-01'), **kwargs)], 
    coords, config, save_data
)

# Function that sends a request for an satellite image of a scene and timestamp 
sentinel_request = lambda eval_script, coords, time_interval, config, save_data, data_folder, **kwargs: send_request(
    eval_script, 
    [SentinelHubRequest.input_data(data_collection=DataCollection.SENTINEL2_L2A, time_interval=time_interval, **kwargs)],
    coords, config, save_data, data_folder
)



Succefully connected!


In [ ]:
# eval_scripts folder

evalscript_true_color = """
    //VERSION=3

    function setup() {
        return {
            input: [{ bands: ["B02", "B03", "B04"] }],
            output: { bands: 3 }
        };
    }

    function evaluatePixel(sample) {
      return [2.5 * sample.B04, 2.5 * sample.B03, 2.5 * sample.B02];
    }
"""

evalscript_ndvi_values = """

//VERSION=3
    function setup() {
      return{
        input: [{
          bands: ["B04", "B08"]
        }],
        output: {
          id: "default",
          bands: 1,
          sampleType: SampleType.FLOAT32
        }
      }
    }

    

    function evaluatePixel(sample) {
      let ndvi = (sample.B08 - sample.B04) / (sample.B08 + sample.B04)
      return [ ndvi ]
    }
"""


In [ ]:
# uncomment the code  below in jupyter notebook and remove uppper cel of python script.
# from python_scripts.connector import * # linked account to Sentinel hub configurations
# from python_scripts.helper import * # conatins functions to creat and send a SentinelhubRequest and plot image

# evalscript for the composite of bands and the value that we want Sentinelhub to return 
# from eval_scripts.evalscript_true_color import *
# from eval_scripts.evalscript_ndvi_values  import *

from sentinelhub import MimeType, CRS, BBox, SentinelHubRequest, SentinelHubDownloadClient,DataCollection, bbox_to_dimensions, SHConfig
from sentinelhub.geo_utils import to_wgs84
from sentinelhub.constants import CRS
from sentinelhub import SentinelHubCatalog

import datetime
import matplotlib.pyplot as plt # visualisation 
import numpy as np 
import pandas as pd
import time # time the performance
import os, shutil 
from PIL import Image, ImageEnhance
import json

The below cell will contains all the functions and other code that has to go into a python script adn imported in to the notebook. 

In [ ]:
# Python Script

def get_time_slots(start, end, chunks):
    """
    Get the avalable time windows (time slots)
    """ 
    datetime_start = datetime.datetime.strptime(start, '%Y-%m-%d')
    datetime_end   = datetime.datetime.strptime(end, '%Y-%m-%d')
    
    tdelta = (datetime_end - datetime_start) / chunks
    edges = [(datetime_start + i*tdelta).date().isoformat() for i in range(chunks)]
    slots = [(edges[i], edges[i+1]) for i in range(len(edges)-1)]
    return slots

def bbox_converter(x):
    """
    Convert coordinates (longitude, latitude) in to a bbox in WGS84 format.
    
    return [long, lat, long, lat]
    """
    lng, lat = to_wgs84(x['Longitude'], x['Latitude'], CRS.WGS84)
    coords = [lng -  0.025*3, lat - 0.025*3, lng +  0.022*3, lat + 0.022*3]
    return coords

def least_clouds():
    return { 
    "dataFilter": { 
        "maxCloudCoverage": 0
        } 
    }

def empty_folder(folder):
    """
    Delete all files in a folder 
    """
    for filename in os.listdir(folder):
        file_path = os.path.join(folder, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e)) 

# creates a SentinelhubRequest in order to download satellite images for a given are and time window. 
def get_request_(evalscript, time_interval, coords, data_folder):
    return SentinelHubRequest(
        data_folder=data_folder, 
        evalscript=evalscript, 
        input_data=[
        SentinelHubRequest.input_data(
            data_collection=DataCollection.SENTINEL2_L2A, 
            time_interval=time_interval,
            other_args = least_clouds())], 
        bbox=BBox(coords,crs=CRS.WGS84), 
        config=config, 
        size=[512, 512], 
        responses=[SentinelHubRequest.output_response('default', MimeType.PNG)])
    
def download_images(geolocations, evalscript, slots, data_folder):
  """
  Download satellite images for geolocations, timestamps and for a composite bands. These imags will be saved in a datafolder.
  """
  for index, row in geolocations.iterrows(): # iterate through each location
      # create a list of requests
      coords = bbox_converter(row) # convert each location to a bbox
      print(coords)
      list_of_requests = [get_request_(evalscript,slot, coords,data_folder) for slot in slots]
      list_of_requests = [request.download_list[0] for request in list_of_requests]
      
      # download data with multiple threads
      SentinelHubDownloadClient(config=config).download(list_of_requests, max_threads=8)

def generate_mask(coords, slot, threshold, data_folder):
    img_ndvi = sentinel_request(evalscript_ndvi_values, coords,slot, config, False,data_folder, other_args=least_clouds())
    min_ndvi = threshold
    ndvi_copy = img_ndvi.copy()
    img_arr = np.where(ndvi_copy > min_ndvi, 255, 0) # categorize ndvi values into binary values/labels: 1(forest) and 0(not forest)
    img_arr = img_arr.astype(np.uint8)
    return img_arr

def download_mask_v2(geolocations, slots, threshold, data_folder):
  """
  Generate NDVI masks using Thresholding with a given threshold.
  """
  for index, rows in geolocations.iterrows(): 
      coords = bbox_converter(rows) # convert each location to a bbox
      for slot in slots:
          labels = generate_mask(coords, slot, threshold, data_folder)
          img = Image.fromarray(labels) # convert labels into image (masks)
          filename = "{}_{}_{}_{}_{}_{}".format(coords[0], coords[1], coords[2],coords[3], slot[0], slot[1])
          img.save(f'{data_folder}/{filename}.png') # save image/mask

def parse_json(path):
    input_json = open(path)
    pased_json = json.load(input_json)
    return pased_json

def build_filename(json):
  """
  Extract features from json file and rename image after features.
  """
  item =  json['payload']['input']
  bbox = item['bounds']['bbox']
  date_start= item['data'][0]['dataFilter']['timeRange']['from'].split("T")[0]
  date_end  = item['data'][0]['dataFilter']['timeRange']['to'].split("T")[0]

  return "{}_{}_{}_{}_{}_{}".format(bbox[0], bbox[1], bbox[2],bbox[3], date_start, date_end) 

def images_structure(rootdir,new_rootdir):
  """
  Extract and rename images and move image to another folder
  """
  for root, subdirectories, files in os.walk(rootdir):
      for subdirectory in subdirectories: # for each folder in dir:
          request_json = f'{rootdir}/{subdirectory}/request.json'
          response_png = f'{rootdir}/{subdirectory}/response.png'
          if os.path.isfile(request_json) and os.path.isfile(response_png):
              json_file = parse_json(request_json)
              new_filename = f'{new_rootdir}/{build_filename(json_file)}.png'
              shutil.copyfile(response_png,new_filename)

# when created, you can import everything in the notebook.
# script name: collect_data_backend.py
# from collect_data_backend import *


In [ ]:
# connecting to google drive 
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Step 1: Get the necessary timestamps

We wanted to meausure deforestation by looking yearly decrease in afforestation across multiple regions in West Bengal India. Senintinel L2A data are avalaible since october 2016. Thus we chose timewindow from 2017 to 2020, with 2020 will be range from (01-01-2020 - 31-12-2020).

https://docs.sentinel-hub.com/api/latest/data/sentinel-2-l2a/



In [ ]:
slots = get_time_slots('2017-01-01', '2021-01-01', 4)
print("Yearly time windows")
for slot in slots:
    print(slot)

Yearly time windows
('2017-01-01', '2018-01-01')
('2018-01-01', '2019-01-01')
('2019-01-01', '2020-01-01')


## Step 2: Get the necessary coordinates 
We needto import the external datasets, which contains the coordinates of the the farmers that Capgemini works with in West bengal. The coordinates are in WGS884 and they're in decimel degrees.

In [ ]:
# convert coordinations_file (csv file) into a dataframe
path_farmerslocations = '../data/locations/converted.csv'
coordinations_df = pd.read_excel('/content/drive/My Drive/BB8/FarmerLocationExtract4Interns_test6.xlsx')

In [ ]:
geolocations  = coordinations_df[['Longitude', 'Latitude']].copy()

In [ ]:
# display the first 3 rows/coordinates 
geolocations.head(3)

,Longitude,Latitude
0,83.143071,21.219172
1,83.363071,21.219172
2,83.583071,21.219172


## Step 3: Downloading satellite images and masks

Now we have to download all the images that we need. We will achieve this with three functions: *empty_folder*, *download_images* and *download_mask_v2*.
*   download_images: download satellite images from different timestamps and multiple spectral emissions using SentinelHub.
*   download_masks: generating NDVI masks using Thresholding with a global threshold of 0.55.



In [ ]:
# remove all files form these folders
#below are the folders 
path_true_colour_images = '../data/images_completed_png' # contains the folders with the images and the json file from SentinelHub 
path_true_colours_images_testset= '../data/true_colours_images_testset_v2' # contains the true colour images.
path_mask_thresholding = '../data/mask_completed' # contains the NDVI mask from download_mask_v2

# empty_folder(path_true_colour_images)
# empty_folder( path_true_colours_images_testset)
# empty_folder(path_mask_thresholding)

empty_folder('/content/drive/My Drive/BB8/West_Bengal_dataset_v3/images')
empty_folder('/content/drive/My Drive/BB8/West_Bengal_dataset_v3/images_enhanced')
empty_folder('/content/drive/My Drive/BB8/West_Bengal_dataset_v3/mask_ndvi_threshold')


In [ ]:
# acquiring the true colour (RGB) satellite images 
start = time.perf_counter()
download_images(geolocations,evalscript_true_color, slots, '/content/drive/My Drive/BB8/West_Bengal_dataset_v3/images') # change path to path_true_colour_images
end = time.perf_counter()
print("Done in: {} seconds".format( end - start))

[83.06807099999999, 21.144172, 83.209071, 21.285172]
[83.28807099999999, 21.144172, 83.429071, 21.285172]
[83.50807099999999, 21.144172, 83.64907099999999, 21.285172]
[83.72807099999999, 21.144172, 83.86907099999999, 21.285172]
[83.94807099999998, 21.144172, 84.08907099999999, 21.285172]
[84.16807099999998, 21.144172, 84.30907099999999, 21.285172]
[84.38807099999998, 21.144172, 84.52907099999999, 21.285172]
[84.60807099999998, 21.144172, 84.74907099999999, 21.285172]
[84.82807099999998, 21.144172, 84.96907099999999, 21.285172]
[85.04807099999998, 21.144172, 85.18907099999998, 21.285172]
[85.26807099999998, 21.144172, 85.40907099999998, 21.285172]
[85.48807099999998, 21.144172, 85.62907099999998, 21.285172]
[85.70807099999998, 21.144172, 85.84907099999998, 21.285172]
[85.92807099999997, 21.144172, 86.06907099999998, 21.285172]
[86.14807099999997, 21.144172, 86.28907099999998, 21.285172]
[86.36807099999997, 21.144172, 86.50907099999998, 21.285172]
[86.58807099999997, 21.144172, 86.729070

In [ ]:
# acquiring the NDVI masks 
start = time.perf_counter()
download_mask_v2(geolocations, slots,0.55, '/content/drive/My Drive/BB8/West_Bengal_dataset_v3/mask_ndvi_threshold') # chang path to path_mask_thresholding
end = time.perf_counter()
print("Done in: {} seconds".format( end - start))

Done in: 480.29134333100046 seconds


## Step 4: Get and rename images

The images that are downloaded from sentinelhub comes in a folder with the image labeled *response.png* and a request json file.

we wil extract the images from the folder and we wil rename the after the following characterestics that we would extract from the request json file:

*bounding box_time window* or <br>
*min Longitude_min Latitude_max Longitude_max Latitude_date_start_date_end*

lastly we wil move these images to another folder.We wil upload the folders to OneDrive.

In [ ]:
rootdir_true_colors = '/content/drive/My Drive/BB8/West_Bengal_dataset_v3/images'
new_rootdir_true_colors = '/content/drive/My Drive/BB8/West_Bengal_dataset_v3/images'
images_structure(rootdir_true_colors, new_rootdir_true_colors)

In [ ]:
rootdir = '/content/drive/My Drive/BB8/West_Bengal_dataset_v3/images'
for filename in os.listdir(rootdir):
  if filename.endswith('.png'):
    im = Image.open(f'{rootdir}/{filename}')
    sharpness = ImageEnhance.Sharpness(im)
    brightness= ImageEnhance.Brightness(im)
    img = sharpness.enhance(3)
    img = brightness.enhance(2)
    img.save(f'/content/drive/My Drive/BB8/West_Bengal_dataset_v3/images_enhanced/{filename}.png')

## Test Cases

In [ ]:
import unittest 

In [35]:
class Test(unittest.TestCase):


  def test_get_time_slot(self):
    actuele_tijdstvensters = [
    ('2017-01-01', '2017-01-07'),
    ('2017-01-07', '2017-01-13'),
    ('2017-01-13', '2017-01-19'),
    ('2017-01-19', '2017-01-25')]
    verwachte_tijdsvensters = get_time_slots('2017-01-01','2017-01-31', 5)
    self.assertEqual(verwachte_tijdsvensters, actuele_tijdstvensters)

   
  def test_bbox_converter(self):
    actuele_bounding_box = [83.06807099999999,21.144172,83.209071,21.285172]
    coordinations_df = pd.read_excel('/content/drive/My Drive/BB8/FarmerLocationExtract4Interns_test6.xlsx')
    geolocations  = coordinations_df[['Longitude', 'Latitude']].copy()
    verwachte_bounding_box = bbox_converter(geolocations.iloc[0])
    self.assertEqual(verwachte_bounding_box, actuele_bounding_box)
  
  # def test_generate_mask(self):
  #   # get mask from the Amazon RainForest Data set 
  #   # Amazon_720.tiff_40.tiff
  #   coordinates = [-55.1953125, -12.487397424166666, -55.14817974277778, -12.441361218611112]
  #   tijdvensters = [
  #   ('2017-01-01', '2017-01-07'),
  #   ('2017-01-07', '2017-01-13'),
  #   ('2017-01-13', '2017-01-19'),
  #   ('2017-01-19', '2017-01-25')]
  #   actuele_mask = np.asarray( Image.open('/content/drive/My Drive/BB8/Amazon Forest Dataset/Training/masks/Amazon_720.tiff_40.png') )
  #   verwachte_mask = generate_mask(coords = coordinates, slot= tijdvensters[3],threshold= 0.55,data_folder= '/content/drive/My Drive/BB8/junk_images')

  #   self.assertEqual(verwachte_mask.all(), actuele_mask.all())
    
  def test_build_filename(self):
    actuele_filename = '83.06807099999999_21.144172_83.209071_21.285172_2017-01-01_2018-01-01.png.png'
    verwachte_filename = build_filename(parse_json('/content/drive/My Drive/BB8/West_Bengal_dataset_v3/images/0a994d7706ae966adbb39fc7161faa60/request.json'))
    
    self.assertEqual(verwachte_filename, actuele_filename)


unittest.main(argv=[''], defaultTest='Test', verbosity=2, exit=False)

test_bbox_converter (__main__.Test) ... ok
test_build_filename (__main__.Test) ... ok
test_get_time_slot (__main__.Test) ... ok

----------------------------------------------------------------------
Ran 3 tests in 0.022s

OK


In [55]:
class TestHelper(unittest.TestCase):


  def test_sentinel_request(self): # test with Amazon rainForest 

      tijdvensters = [
      ('2017-01-01', '2018-01-01'),
      ('2018-01-01', '2019-01-01'),
      ('2019-01-01', '2020-01-01')]
      actuele_img = np.asarray( Image.open('/content/drive/My Drive/BB8/Amazon Forest Dataset/Training/images/png/Amazon_720.png') )
      coordinates = [-55.1953125, -12.487397424166666, -55.14817974277778, -12.441361218611112]
      verwachte_img = sentinel_request(evalscript_true_color, coordinates, tijdvensters[1], config, False,'/content/drive/My Drive/BB8/junk_images', other_args=least_clouds())

      self.assertAlmostEqual(verwachte_img.all(), actuele_img.all())
    
unittest.main(argv=[''], defaultTest='TestHelper', verbosity=2, exit=False)

test_sentinel_request (__main__.TestHelper) ... ok

----------------------------------------------------------------------
Ran 1 test in 3.919s

OK
